In [20]:
import requests
from  bs4 import BeautifulSoup


In [8]:
pip install selenium

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/0e/59/aae37fa93e2d4292c3148efcc3066c8ecfe5cfaa72bf8c0b1a5614622cf7/selenium-4.15.2-py3-none-any.whl.metadata
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/39/46/620fbe56f41fa3ccdda2136d947fb9bacce3d1eb163f057f0262a0ddf5e0/trio-0.23.1-py3-none-any.whl.metadata
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/48/be/a9ae5f50cad5b6f85bd2574c2c923730098530096e170c1ce7452394d7aa/trio_websocket-0.11.1-py3-none-any.whl.metadata
  Obtaining dependency information for outcome from https://files.pythonhosted.org/packages/55/8b/5ab7257531a5d830fc8000c476e63c935488d74609b50f9384a643ec0a62/outcome-1.3.0.post0-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/58.3 kB ? eta -:--:--
     ---------------


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

def create_firefox_webdriver(headless=False):
    # Set options for WebDriver
    options = Options()
    options.headless = headless

    # Create a new instance of the Firefox driver
    driver = webdriver.Firefox(options=options)

    return driver

if __name__ == "__main__":
    # Example usage
    driver = create_firefox_webdriver(headless=True)  # Set headless=True if you don't need a GUI
    driver.get("https://www.makemytrip.com/hotels/hotel-listing/?checkin=12062023&checkout=12082023&locusId=CTAYA&locusType=city&city=CTAYA&country=IN&searchText=Ayodhya&roomStayQualifier=2e0e&_uCurrency=INR&reference=hotel&type=city&rsc=1e2e0e")
    print(driver.title)  # Prints the title of the page
    driver.quit()


Example Domain


In [17]:
driver = create_firefox_webdriver(headless=True)  # Set headless=True if you don't need a GUI
driver.get("https://www.makemytrip.com/hotels/hotel-listing/?checkin=12062023&checkout=12082023&locusId=CTAYA&locusType=city&city=CTAYA&country=IN&searchText=Ayodhya&roomStayQualifier=2e0e&_uCurrency=INR&reference=hotel&type=city&rsc=1e2e0e")
print(driver.title)  # Prints the title of the page

Ayodhya Hotels, resorts, homestays and more | MakeMytrip.com


In [22]:
soup = BeautifulSoup(driver.page_source)

In [59]:
all_hotels = soup.find_all('div',{'class':'listingRowOuter'})

In [60]:
len(all_hotels)

103

In [61]:
import re

def extract_prices(text):
    # Regular expression to find the prices
    pattern = r'₹ (\d{1,3}(?:,\d{3})?|d+)₹ (\d{1,3}(?:,\d{3})?|\d+)(?:\+ ₹ (\d{1,3}(?:,\d{3})?|\d+) taxes)?'

    # Find all matches
    matches = re.findall(pattern, text)

    # Convert matches to a list of dictionaries
    cleaned_data = []
    for match in matches:
        cleaned_data.append({
            'base_price': match[0],
            'current_price': match[1],
            'tax': match[2]
        })

    return cleaned_data[0]

In [66]:
import pandas as pd

hotels_in_ayodhya_from_mmt = pd.DataFrame()

for hotel in all_hotels:
    row = {}
    link = hotel.find('a')['href']
    print(link)
    hotel_name = hotel.find('p',{'id':'hlistpg_hotel_name'}).text
    print(hotel_name)
    row['Hotel_Name'] = hotel_name
    presuation_text = hotel.find('div',{'class':'addrContainer'}).text.strip()
    print(presuation_text)
    if '|' in presuation_text:
        location = presuation_text.split('|')[0]
        relative_address = presuation_text.split('|')[1]
    else:
        location = presuation_text
        relative_address = 'Need to add'
    row['location']=location
    row['relative_address']=relative_address   
    price_details = hotel.find('div',{'class':'textRight priceDetails'})

    
    prices = extract_prices(price_details.text)
    # print(prices)
    # print(price_details.text)
    row['base_price'] = prices.get('base_price','Not Available')
    row['current_price'] = prices.get('current_price','Not Available')
    row['tax'] = prices.get('tax','Not Available')
    row['link'] = link
    print(row, prices)

    df = pd.DataFrame([row])
    frame = [hotels_in_ayodhya_from_mmt,df]
    hotels_in_ayodhya_from_mmt = pd.concat(frame)

//www.makemytrip.com/hotels/hotel-details?hotelId=202310161917097917&_uCurrency=INR&checkin=12062023&checkout=12082023&city=CTAYA&country=IN&lat=26.80134&lng=82.20409&locusId=CTAYA&locusType=city&rank=1&reference=hotel&roomStayQualifier=2e0e&rsc=1e2e0e&searchText=Ayodhya&type=city&mtkeys=7137147783883716611
Apoorva Guest House
Tulsi Nagar
{'Hotel_Name': 'Apoorva Guest House', 'location': 'Tulsi Nagar', 'relative_address': 'Need to add', 'base_price': '2,000', 'current_price': '1,682', 'tax': '366', 'link': '//www.makemytrip.com/hotels/hotel-details?hotelId=202310161917097917&_uCurrency=INR&checkin=12062023&checkout=12082023&city=CTAYA&country=IN&lat=26.80134&lng=82.20409&locusId=CTAYA&locusType=city&rank=1&reference=hotel&roomStayQualifier=2e0e&rsc=1e2e0e&searchText=Ayodhya&type=city&mtkeys=7137147783883716611'} {'base_price': '2,000', 'current_price': '1,682', 'tax': '366'}
//www.makemytrip.com/hotels/hotel-details?hotelId=202301301942286805&_uCurrency=INR&checkin=12062023&checkout=12

In [68]:
hotels_in_ayodhya_from_mmt.to_excel("Hotels in Ayodhya.xlsx")

In [80]:
hotels_in_ayodhya_from_mmt

,Hotel_Name,location,relative_address,base_price,current_price,tax,link
0,Apoorva Guest House,Tulsi Nagar,Need to add,"2,000","1,682",366,//www.makemytrip.com/hotels/hotel-details?hote...
0,Hotel Hanuman Ji,Professor's Colony,Need to add,"1,599","1,116",284,//www.makemytrip.com/hotels/hotel-details?hote...
0,OYO Home Siya Homestay,Faizabad,11.8 km from city centre,"4,052","2,269",,//www.makemytrip.com/hotels/hotel-details?hote...
0,Hotel Shri Ram Bhavan Seeta Swayamvar Marriage...,Ayodhya,Need to add,"1,650","1,535",363,//www.makemytrip.com/hotels/hotel-details?hote...
0,Life Point grand,Faizabad,9.7 km from city centre,"2,400","2,232",456,//www.makemytrip.com/hotels/hotel-details?hote...
...,...,...,...,...,...,...,...
0,OYO Home Kanak Hotel & Resort Homestay,Ismailpur,Need to add,"2,912","1,392",84,//www.makemytrip.com/hotels/hotel-details?hote...
0,Yashmay Villa Club & Resort,Gonda,Need to add,"4,600","4,278","1,012",//www.makemytrip.com/hotels/hotel-details?hote...
0,Mosaic Hotel and Restaurant,Gonda,Need to add,999,722,180,//www.makemytrip.com/hotels/hotel-details?hote...
0,"Hotel White Stone, Gonda",Gonda,Need to add,"2,295","1,814",429,//www.makemytrip.com/hotels/hotel-details?hote...


In [70]:
links = hotels_in_ayodhya_from_mmt['link'].tolist()
links[0]

'//www.makemytrip.com/hotels/hotel-details?hotelId=202310161917097917&_uCurrency=INR&checkin=12062023&checkout=12082023&city=CTAYA&country=IN&lat=26.80134&lng=82.20409&locusId=CTAYA&locusType=city&rank=1&reference=hotel&roomStayQualifier=2e0e&rsc=1e2e0e&searchText=Ayodhya&type=city&mtkeys=7137147783883716611'

In [86]:
import time
link = links[0][2:]
link
for i, row in hotels_in_ayodhya_from_mmt.iterrows():
    link = row['link']
    driver.get('https://'+link)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source)
    image_tags = soup.find_all('img')
    
    # Extract src attribute for each image tag
    image_urls = [img['src'] for img in image_tags if 'src' in img.attrs]
    image_urls = [x for x in image_urls if x.startswith("//")]

    def save_image(url, filename):
        response = requests.get(url)
        if response.status_code == 200:
            with open(filename, 'wb') as file:
                file.write(response.content)

    # Directory to save the images
    directory = 'hotel_images/' + row['Hotel_Name']
    
    os.makedirs(directory, exist_ok=True)
    
    # Download and save each image
    for i, url in enumerate(image_urls):
        filename = os.path.join(directory, f'hotel_image_{i}.jpg')
        save_image('https:'+url, filename)

['//imgak.mmtcdn.com/pwa_v3/pwa_hotel_assets/header/logo@2x.png', '//r2imghtlak.mmtcdn.com/r2-mmt-htl-image/htl-imgs/202310161917097917-118f598c6c2711eeb8ec0a58a9feac02.jpg?&output-quality=75&downsize=520:350&crop=520:350;2,0&output-format=jpg', '//r2imghtlak.mmtcdn.com/r2-mmt-htl-image/htl-imgs/202310161917097917-11cad2006c2711ee88b40a58a9feac02.jpg?&output-quality=75&downsize=520:350&crop=520:350;2,0&output-format=jpg', '//r2imghtlak.mmtcdn.com/r2-mmt-htl-image/htl-imgs/202310161917097917-10eedb066c2711ee82cd0a58a9feac02.jpg?&output-quality=75&downsize=520:350&crop=520:350;2,0&output-format=jpg', '//r2imghtlak.mmtcdn.com/r2-mmt-htl-image/htl-imgs/202310161917097917-12ef49726c2711ee8bbe0a58a9feac02.jpg?&output-quality=75&downsize=520:350&crop=520:350;2,0&output-format=jpg', '//r2imghtlak.mmtcdn.com/r2-mmt-htl-image/htl-imgs/202310161917097917-1131b4946c2711eeafb00a58a9feac02.jpg?&output-quality=75&downsize=520:350&crop=520:350;2,0&output-format=jpg', '//r2imghtlak.mmtcdn.com/r2-mmt-ht

In [79]:
import requests
import os

def save_image(url, filename):
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, 'wb') as file:
            file.write(response.content)

# Directory to save the images
directory = 'hotel_images'
os.makedirs(directory, exist_ok=True)

# Download and save each image
for i, url in enumerate(image_urls):
    filename = os.path.join(directory, f'hotel_image_{i}.jpg')
    save_image('https:'+url, filename)


In [ ]:
import time
link = links[0][2:]
link
for i, row in hotels_in_ayodhya_from_mmt.iterrows():
    link = row['link']
    driver.get('https://'+link)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source)
    image_tags = soup.find_all('img')
    
    # Extract src attribute for each image tag
    image_urls = [img['src'] for img in image_tags if 'src' in img.attrs]
    image_urls = [x for x in image_urls if x.startswith("//")]

    def save_image(url, filename):
        response = requests.get(url)
        if response.status_code == 200:
            with open(filename, 'wb') as file:
                file.write(response.content)

    # Directory to save the images
    directory = 'hotel_images/' + row['Hotel_Name']
    
    os.makedirs(directory, exist_ok=True)
    
    # Download and save each image
    for i, url in enumerate(image_urls):
        filename = os.path.join(directory, f'hotel_image_{i}.jpg')
        save_image('https:'+url, filename)